# **Tratamento e coleta de informações de inventário de equipamentos de rede IP: roteadores e switches Cisco**

#### 1) Este código realiza a conversão do inventário de equipamentos de rede do formato semi-estruturado, extraída através da interface de linha de comando dos equipamentos, para o formato de tabela.

#### 2) Além disso, extrai a localização dos equipamentos baseado no nome dos mesmos.

#### 3) E por fim, busca e acrescenta informações relativas ao ciclo de vida do equipamento, como se já encontra-se obsoleto (fabricante não dá mais suporte) e se tem algum contrato de suporte associado. 


## Instalação de biblioteca

In [ ]:
!pip install textfsm
!pip install pandas
!pip install regex
import  textfsm
import pandas as pd
import regex as re
import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import json


## 1) Conversão do texto extraído de comando na CLI para formato de tabela

Adequação do arquivo fonte com "inventário" dos equipamentos, inserindo o  marcador do prompt de comando para ficar de acordo com template utilizado. Inserido # após o nome do equipamento.

In [ ]:
old_input_file = open("show_inventory_real.txt", encoding='utf-8')
input_file = open("show_inventory_real_mod.txt","wt")
for line in old_input_file:
        input_file.write(line.replace(" | CHANGED |","#"))
old_input_file.close()
input_file.close()

Atribuição do conteúdo do arquivo de inventário modificado a uma variável.

In [ ]:
input_file = open("show_inventory_real_mod.txt", encoding='utf-8')
raw_text_data = input_file.read()
input_file.close()

*Parser* do texto com FSM. O template é utilizado para comparar o conteúdo do texto com o invetory dos equipamentos e localizar cada informação.

In [ ]:
template = open("show_inventory_multiple.textfsm")
re_table = textfsm.TextFSM(template)
fsm_results = re_table.ParseText(raw_text_data)





Criação de dataframe com a lista de equipamentos.

In [ ]:
df_equip = pd.DataFrame(fsm_results)

In [ ]:
df_equip





Inserindo título das colunas

In [ ]:
df_equip.columns = re_table.header

In [ ]:
df_equip.head()

## 2) Extração da informação de localização do equipamento

Esta parte extrai informação de localização do equipamento (site) baseado no hostname. 

In [ ]:
df = df_equip
df.dtypes

In [ ]:
df['site'] = df['hostname'].str.findall('([a-zA-Z ]+)$|([a-zA-Z]+[0-9]+)$')
df

In [ ]:
#Como o nome do site apareceu entre parênteses e colchetes, formando uma tupla com elemento vazio, 
#foi necessário criar uma função para extrair a informação somente a informação do site.
def get_items(site_row):
  list_item = site_row[0]
  item = [x for x in list_item if x != ""]
  return item[0]

def get_site(row):
  site = row['site']
  item = get_items(site)
  row['Site'] = item
  return row
df2 = df.apply(get_site, axis=1)

In [ ]:
df2

In [ ]:
#Excluindo as colunas que não necessárias
df3 = df2.drop(columns =['site','vid'])
df3

## 3) Localização de informações relacionadas ao ciclo de vida do equipamento junto ao fabricante.

Etapa de autenticação e atribuição de token no site do fabricante

In [ ]:
def get_token():
    headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    }
    data = {
     'client_id': 'inserir ID',
     'client_secret': 'inserir secret',
     'grant_type': 'client_credentials'
    }
    response = requests.post('https://cloudsso.cisco.com/as/token.oauth2', headers=headers, data=data, verify=False)
    js = response.json()
    token = (js['access_token'])
    headers = {
        'Accept': 'application/json',
        'Authorization': 'Bearer ' + token,
    }
    return headers

In [ ]:
var_token = get_token()

Consulta via API no site do Fabricante

In [ ]:
# definição de função para consulta se o equipamento está coberto com algum contrato
def get_coverage(num_serie):
    if (num_serie == "N/A"):
        coverage = "Not Available"
    else:
     response1 = requests.get('https://api.cisco.com/sn2info/v2/coverage/status/serial_numbers/%s' %num_serie, headers=var_token, verify=False)
     data1 = json.loads(response1.text)
     coverage = data1['serial_numbers'][0]['is_covered']
    return coverage

In [ ]:
# definição de função para consulta de última data de suporte do produto associado ao número de série informado.
def get_eos(produtoid):
    if (produtoid == 'Unspecified'):
        EndOfSupport = "Not Available"
    else:
        response2 = requests.get('https://api.cisco.com/supporttools/eox/rest/5/EOXByProductID/%s' %produtoid, headers=var_token, verify=False)
        data2 = json.loads(response2.text)
        DataEndOfSupport = data2['EOXRecord'][0]['LastDateOfSupport']['value']
        if (DataEndOfSupport != ''): 
            EndOfSupport = DataEndOfSupport
        else:
            EndOfSupport = "Not Announced"  
    return EndOfSupport

In [ ]:
df3['coverage']=''
df3

In [ ]:
df3.dtypes

In [ ]:
#Criada função para consulta da cobertura contratual, prevendo que seja inserido 
#"erro" quando a consulta não for viável, seja por token inválido, seja por 
#consulta de parêmetro inválido.
def query_coverage(row):
    try:
      contrato_row = row['coverage']
      serialNumber_row = row['serialnumber']
      if contrato_row == '':
        row['coverage'] = get_coverage(serialNumber_row)
      return row
    except:
      row['coverage'] = 'Erro'
      return row 

In [ ]:
df4 = df3.apply(query_coverage, axis=1)

In [ ]:
df4

In [ ]:
df4['EoS']=''

In [ ]:
df4

In [ ]:
#Criada função para consulta do fim de suporte, prevendo que seja inserido 
#"erro" quando a consulta não for viável, seja por token inválido, seja por 
#consulta de parêmetro inválido.
def query_eos(row):
    try: 
      eos_row = row['EoS']
      productid_row = row['productid']
      if eos_row == '':
        row['EoS'] = get_eos(productid_row)
      return row
    except:
      row['EoS'] = 'Erro'
      return row

In [ ]:
df5 = df4.apply(query_eos, axis=1)

In [ ]:
df5

In [ ]:
df5.to_csv('Dados_Equipamentos.csv', index=False)

## **Referências:**


* http://yang.ciscolive.com/pod0/labs/lab9/lab9-m8
* https://code.google.com/archive/p/textfsm/
* https://codingnetworker.com/2015/08/parse-cli-outputs-textfsm/

